In [1]:
import numpy as np
import pandas as pd
df = pd.read_csv('results.txt')
df

,file,label,aegypti,albopictus,anopheles,culex,culiseta,japonicus_koreicus,other_insect,other_mosquito,unidentifiable
0,/home/ubuntu/vol/inference_cropped_dataset/plu...,anopheles,0.000730,0.007150,0.750100,0.027917,0.002765,0.008269,0.156652,0.023345,0.023072
1,/home/ubuntu/vol/inference_cropped_dataset/plu...,anopheles,0.001196,0.004946,0.914370,0.000598,0.002534,0.004381,0.031842,0.002828,0.037305
2,/home/ubuntu/vol/inference_cropped_dataset/plu...,anopheles,0.003512,0.025274,0.923695,0.001796,0.001970,0.010824,0.027991,0.003858,0.001079
3,/home/ubuntu/vol/inference_cropped_dataset/plu...,anopheles,0.016142,0.064402,0.005710,0.274745,0.091685,0.015240,0.020373,0.500746,0.010957
4,/home/ubuntu/vol/inference_cropped_dataset/plu...,anopheles,0.001211,0.015076,0.378325,0.138276,0.008089,0.003427,0.219319,0.033289,0.202988
...,...,...,...,...,...,...,...,...,...,...,...
21997,/home/ubuntu/vol/inference_cropped_dataset/Pro...,culex,0.000661,0.001538,0.008908,0.507307,0.084463,0.000665,0.118581,0.007421,0.270457
21998,/home/ubuntu/vol/inference_cropped_dataset/Pro...,culex,0.000070,0.000179,0.000432,0.990694,0.000493,0.000046,0.000316,0.000178,0.007591
21999,/home/ubuntu/vol/inference_cropped_dataset/Pro...,culex,0.005632,0.058240,0.001601,0.424649,0.015594,0.001327,0.222104,0.023720,0.247134
22000,/home/ubuntu/vol/inference_cropped_dataset/Pro...,culex,0.000010,0.000033,0.000095,0.998239,0.000690,0.000008,0.000105,0.000079,0.000740


In [7]:
with open('scores.dzn','w+') as rocfile:                
    for specie in species_counts:
        print(f"{specie} = array2d(1..{species_counts[specie]}, 1..9,\n[",file=rocfile)
        for nsk, row in df[df['label']==specie].iterrows():
            print(','.join(arrodonir(row.iloc[2:]))+',',file=rocfile) # TODO: remove last comma from each array
        print("]);",file=rocfile)

In [2]:
species_counts = dict(df.label.value_counts())
species_counts

{'albopictus': 15658,
 'culex': 4694,
 'culiseta': 682,
 'other_mosquito': 580,
 'japonicus_koreicus': 175,
 'anopheles': 82,
 'aegypti': 77,
 'complex': 52,
 'conflict': 2}

In [5]:
def arrodonir(llista):
    return list(map(lambda x: str(int(100*round(x,2))),llista))
def printSpeciesIndex():
    print("""
int: i_aegypti = 1;
int: i_albopictus = 2;
int: i_anopheles = 3;
int: i_culex = 4;
int: i_culiseta = 5;
int: i_japonicus_koreicus =6;
int: i_complex = 7;
int: i_other_mosquito = 8;
int: i_conflict = 9;
""")
smooth = {}
total = len(df)
smoothing_factor = 5
for specie in species_counts:
    smooth[specie] = 100000*(smoothing_factor+species_counts[specie]) / (total + 9*smoothing_factor)
smooth

{'albopictus': 71043.67941216491,
 'culex': 21313.557400099788,
 'culiseta': 3116.070213634508,
 'other_mosquito': 2653.4222343175943,
 'japonicus_koreicus': 816.4376105592597,
 'anopheles': 394.6115117703089,
 'aegypti': 371.9326892547739,
 'complex': 258.53857667709894,
 'conflict': 31.75035152174899}

In [6]:
with open('thresholds_.mzn','w+') as rocfile:
    print("array[1..9] of var 0..100: thresholds;")
    for specie in species_counts:
        print(f"int: n_{specie} = {species_counts[specie]};")
    print(f"int: n_total = {'+'.join(map(lambda x: 'n_'+x, species_counts))};")
    for specie in species_counts:
        print(f"int: factor_{specie} = {int(smooth[specie])};")
    printSpeciesIndex()
    for specie in species_counts:
        print(f"array[1..n_{specie},1..9] of var 0..100: {specie};")
    fn_pairs = {}
    for specie in species_counts:
        fn_pairs[specie] = []
        print(f"var 0..n_{specie}: tp_{specie} = count([{specie}[i,i_{specie}] >= thresholds[i_{specie}] | i in 1..n_{specie}]);")
        print(f"var 0..100: tpr_{specie} = 100* tp_{specie} div n_{specie};")
    fp_pairs = {}   
    for specie_i in species_counts:
        fp_pairs[specie_i] = []
        for specie_n in species_counts:
            if specie_i != specie_n:
                fp_pairs[specie_i].append(f"fp_{specie_i}_{specie_n}")
                fn_pairs[specie_n].append(f"fn_{specie_n}_{specie_i}")
                print(f"var int: fp_{specie_i}_{specie_n} = count([{specie_n}[i,i_{specie_i}] >= thresholds[i_{specie_i}] | i in 1..n_{specie_n}]);") 
                print(f"var 0..n_{specie_n}: fn_{specie_n}_{specie_i} = fp_{specie_i}_{specie_n};") 
    for specie_i in species_counts:
        print(f"var int: fp_{specie_i} = {'+'.join(fp_pairs[specie_i])};")
        print(f"var int:fpr_{specie_i} = 100* ({'+'.join(fp_pairs[specie_i])}) div (n_total - n_{specie_i});")
        print(f"var 0..n_{specie_i}: fn_{specie_i} = {'+'.join(fn_pairs[specie_i])};")
        print(f"var 0..100: fnr_{specie_i} = 100*fn_{specie_i} div n_{specie_i};")
        print(f"var 0..n_{specie_i}: nc_{specie_i} = n_{specie_i} - (tp_{specie_i}+fn_{specie_i});")
        print(f"var 0..100: ncr_{specie_i} = 100*nc_{specie_i} div n_{specie_i};")
    for specie in species_counts:
        print(f"constraint forall(n in 1..n_{specie})(count([{specie}[n,i] >= thresholds[i] | i in 1..9])<=1);")
    minimize_factors = {}
    for specie in species_counts:
        print(f"int: min_factor_{specie} = factor_{specie} * (fpr_{specie} + ncr_{specie});")
    print(f"solve minimize(sum({'+'.join(map(lambda x: 'min_factor_'+x, species_counts))}));")
    print("output [\"class \\t \\t total \\t tp \\t fp \\t fn \\t nc \\t tpr \\t fnr \\t ncr \\t fpr \\n\"];")
    for specie in species_counts:
        print(f"output [\"{specie[:5]} \\t \\t \\(n_{specie}) \\t \\(tp_{specie}) \\t \\(fp_{specie}) \\t \\(fn_{specie}) \\t \\(nc_{specie}) \\t \(tpr_{specie}) \\t \(fnr_{specie}) \\t \(ncr_{specie}) \\t \(fpr_{specie}) \\n\"];")
    print("output [\"\(thresholds[i]) \" | i in 1..9];")

array[1..9] of var 0..100: thresholds;
int: n_albopictus = 15658;
int: n_culex = 4694;
int: n_culiseta = 682;
int: n_other_mosquito = 580;
int: n_japonicus_koreicus = 175;
int: n_anopheles = 82;
int: n_aegypti = 77;
int: n_complex = 52;
int: n_conflict = 2;
int: n_total = n_albopictus+n_culex+n_culiseta+n_other_mosquito+n_japonicus_koreicus+n_anopheles+n_aegypti+n_complex+n_conflict;
int: factor_albopictus = 71043;
int: factor_culex = 21313;
int: factor_culiseta = 3116;
int: factor_other_mosquito = 2653;
int: factor_japonicus_koreicus = 816;
int: factor_anopheles = 394;
int: factor_aegypti = 371;
int: factor_complex = 258;
int: factor_conflict = 31;

int: i_aegypti = 1;
int: i_albopictus = 2;
int: i_anopheles = 3;
int: i_culex = 4;
int: i_culiseta = 5;
int: i_japonicus_koreicus =6;
int: i_complex = 7;
int: i_other_mosquito = 8;
int: i_conflict = 9;

array[1..n_albopictus,1..9] of var 0..100: albopictus;
array[1..n_culex,1..9] of var 0..100: culex;
array[1..n_culiseta,1..9] of var 0..10